In [1]:
import os
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [2]:
dataset_directory = '.\\dataset'

path = os.listdir(dataset_directory)

In [3]:
'''
Reading the data using pandas dataframe

'''

labels = pd.read_csv(os.path.join(dataset_directory, 'concepts_2011.txt'), sep="\t")
image_labels = pd.read_csv(os.path.join(dataset_directory, 'trainset_gt_annotations.txt'), sep=" ", header=None)

In [4]:
labels.name[10]

'Summer'

In [5]:
image_labels.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64')

In [6]:
image_label = image_labels.drop(columns = [i for i in range(1,10)])
image_label = image_label.drop(columns = [i for i in range(14,100)])
image_label.columns = ['image', 'spring','summer', 'autumn', 'winter']
image_label

,image,spring,summer,autumn,winter
0,0039b5a7-c1ad-423a-92a0-3f38558043a2.jpg,0,0,0,0
1,003eaf28-898d-404f-abe5-e8e86d422fa2.jpg,0,0,0,0
2,005f4848-780a-4d31-8c09-4abdfd46804c.jpg,0,0,0,0
3,00a53424-5a0d-4c9d-ac90-242a6099fc35.jpg,0,0,0,0
4,00b5c2a5-2b8f-492b-ad5d-ed3310e6da56.jpg,0,0,0,0
...,...,...,...,...,...
7995,{FFA81171-399C-432E-8373-753B7FC5F85B}.jpg,0,1,0,0
7996,{FFB02EA2-96FB-4C60-ADEF-C57C0CC2266E}.jpg,0,0,0,0
7997,{FFB61D93-7A87-487B-8478-58B78E878823}.jpg,0,0,0,0
7998,{FFC6F015-ABF9-4AD9-ACC9-0941FD14A251}.jpg,0,0,0,0


In [7]:
image_label_summer = image_label[image_label.summer != 0]
image_label_summer.shape

(887, 5)

In [8]:
image_label_spring = image_label[image_label.spring != 0]
image_label_spring.shape

(105, 5)

In [9]:
image_label_autumn = image_label[image_label.autumn != 0]
image_label_autumn.shape

(153, 5)

In [10]:
image_label_winter = image_label[image_label.winter != 0]
image_label_winter.shape

(208, 5)

In [11]:
image_label_data = pd.concat([image_label_spring, image_label_summer, image_label_autumn, image_label_winter]).drop_duplicates()

In [12]:
image_label_data

,image,spring,summer,autumn,winter
16,01d88c87-f2e3-4a7f-be26-b38c549b5e7d.jpg,1,0,0,0
91,0a42fb46-cf0b-4d71-8e4c-8919986071d7.jpg,1,0,0,0
104,0b946594-634d-48df-a270-aa980cd6a0c8.jpg,1,0,0,0
112,0cf35146-61ff-4a6c-975a-5e63b291eb52.jpg,1,0,0,0
187,14ae13f7-c528-4d9d-b421-81fa135168e9.jpg,1,0,0,0
...,...,...,...,...,...
7805,{F6788D91-96BB-4BCA-AD56-FFFADE5C226C}.jpg,0,0,0,1
7829,{F749F3B9-188B-444B-8AAB-F28397D22C6F}.jpg,0,0,0,1
7840,{F8167184-5385-400B-8F04-F970D7646982}.jpg,0,0,0,1
7865,{F935DF04-5661-4897-B2BF-64BF9BC86A98}.jpg,0,0,0,1


In [13]:
x = np.array([np.load(os.path.join(os.path.join(dataset_directory,"imageclef2011_feats") , file + '_ft.npy')) for file in image_label_data.image ])

In [14]:
target = image_label_data.drop(columns = ['image'])

In [15]:
seasons = ['spring', 'summer', 'autumn', 'winter']

'''
Choosing the season
'''
y = target[seasons[0]]
y

16      1
91      1
104     1
112     1
187     1
       ..
7805    0
7829    0
7840    0
7865    0
7931    0
Name: spring, Length: 1353, dtype: int64

In [16]:
'''
60-15-25 split for training, test, validation respectively
'''

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.15, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,test_size=338/1150, stratify=y_train)

In [17]:
x_train.shape, x_test.shape, x_val.shape

((812, 1024), (203, 1024), (338, 1024))

In [18]:
y_train.shape, y_test.shape, y_val.shape

((812,), (203,), (338,))

In [19]:
regularization_constants = [0.01, 0.1, 0.1**0.5, 1, 10**0.5, 10, 100**0.5]

In [22]:
from sklearn.svm import LinearSVC

for i in regularization_constants:
    svm = LinearSVC(random_state=0,C=i)
    svm.fit(x_train,y_train)
    print(svm.score(x_val, y_val))

0.9112426035502958
0.8816568047337278


c:\users\chris\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8816568047337278


c:\users\chris\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8816568047337278


c:\users\chris\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.878698224852071


c:\users\chris\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.878698224852071
0.878698224852071


c:\users\chris\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
